# 📦 ONNX Export Notebook

This notebook exports the trained speaker verification model to **ONNX format** for deployment.

## 🔄 What is ONNX?

**ONNX** (Open Neural Network Exchange) is an open format for representing machine learning models. It allows you to:

1. **Deploy to different platforms**: Run the model in C++, C#, Java, JavaScript, etc.
2. **Use different runtimes**: ONNX Runtime, TensorRT, OpenVINO for optimization
3. **Cross-framework compatibility**: Export from PyTorch, use in TensorFlow or other frameworks
4. **Optimize for inference**: Apply quantization, graph optimizations

## 📊 Export Strategy

We export only the **Backbone** (CNN + RNN) which produces speaker embeddings:

```
┌─────────────────────────────────────────────────────────────────────┐
│                    FULL MODEL (Training)                            │
│  ┌──────────────┐   ┌──────────────┐   ┌──────────────────────────┐│
│  │   Backbone   │ → │  Embeddings  │ → │     AAMSoftmax Head      ││
│  │  (CNN+RNN)   │   │   [B, 256]   │   │  (Only for training)     ││
│  └──────────────┘   └──────────────┘   └──────────────────────────┘│
└─────────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────────┐
│                    EXPORTED MODEL (ONNX)                            │
│  ┌──────────────┐   ┌──────────────┐                               │
│  │   Backbone   │ → │  Embeddings  │  ← This is what we export    │
│  │  (CNN+RNN)   │   │   [B, 256]   │                               │
│  └──────────────┘   └──────────────┘                               │
└─────────────────────────────────────────────────────────────────────┘
```

**Why only the backbone?**
- The AAMSoftmax head is only used during training to learn better embeddings
- At inference time, we compare embeddings directly using cosine similarity
- The speaker bank (reference embeddings) is stored separately

In [ ]:
# ============================================================================
# SECTION 1: IMPORTS AND CONFIGURATION
# ============================================================================

import os
import glob
import warnings
import numpy as np
import h5py
import yaml
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.onnx

warnings.filterwarnings("ignore", category=FutureWarning)

print("="*70)
print("📦 ONNX EXPORT SETUP")
print("="*70)

# ============================================================================
# SELECT CHECKPOINT AND OUTPUT PATH
# ============================================================================

print("\n📁 AVAILABLE CHECKPOINTS:")
checkpoint_dirs = sorted(glob.glob("checkpoints/train*"), key=os.path.getmtime, reverse=True)
for i, cp in enumerate(checkpoint_dirs[:10]):
    best_path = os.path.join(cp, "best_model.pt")
    exists = "✓" if os.path.exists(best_path) else "✗"
    mtime = datetime.fromtimestamp(os.path.getmtime(cp)).strftime("%Y-%m-%d %H:%M")
    print(f"   {i+1}. {cp} [{exists}] ({mtime})")

# ⚠️ CONFIGURE THESE PATHS
checkpoint_path = os.path.join(checkpoint_dirs[0], "best_model.pt") if checkpoint_dirs else "checkpoints/train1/best_model.pt"
onnx_output_path = "./checkpoints/speaker_backbone.onnx"

# Dataset path for metadata
available_datasets = sorted(glob.glob("outputs/logmels_*.h5"), key=os.path.getmtime, reverse=True)
dataset_path = available_datasets[0] if available_datasets else "outputs/logmels_aug.h5"

print(f"\n✅ Selected Checkpoint: {checkpoint_path}")
print(f"✅ ONNX Output: {onnx_output_path}")
print(f"✅ Dataset (for metadata): {dataset_path}")

## Section 2: Model Architecture

Define the model architecture (must match training exactly).

In [ ]:
# ============================================================================
# SECTION 2: MODEL ARCHITECTURE (Same as training)
# ============================================================================

class SEBlock(nn.Module):
    """Squeeze-and-Excitation block."""
    def __init__(self, channels: int, reduction: int = 16):
        super().__init__()
        hidden = max(channels // reduction, 4)
        self.fc = nn.Sequential(
            nn.Linear(channels, hidden, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(hidden, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        B, C, T, F = x.shape
        s = x.mean(dim=(2, 3))
        w = self.fc(s).view(B, C, 1, 1)
        return x * w


class Backbone(nn.Module):
    """CNN + RNN backbone for speaker embeddings."""
    def __init__(self, no_mels, embed_dim, rnn_hidden, rnn_layers, bidir):
        super().__init__()

        self.cnn_block = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            SEBlock(32, reduction=8),
            nn.MaxPool2d(kernel_size=(1, 2)),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            SEBlock(64, reduction=8),
            nn.MaxPool2d(kernel_size=(1, 2)),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            SEBlock(128, reduction=8),
            nn.MaxPool2d(kernel_size=(1, 2)),
        )

        self.rnn_hidden = rnn_hidden
        self.rnn = nn.GRU(
            input_size=128 * (no_mels // 8),
            hidden_size=self.rnn_hidden,
            num_layers=rnn_layers,
            bidirectional=bidir,
            batch_first=True,
            dropout=0.2
        )

        out_dim = (2 if bidir else 1) * rnn_hidden
        self.rnn_ln = nn.LayerNorm(out_dim)

        self.att = nn.Sequential(
            nn.Linear((2 if bidir else 1)*rnn_hidden, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )

        self.proj = nn.Sequential(
            nn.Linear(out_dim*2, 256),
            nn.BatchNorm1d(256), nn.ReLU(),
            nn.Linear(256, embed_dim)
        )

    def forward(self, x, lengths=None, mc_dropout=None):
        if mc_dropout is None:
            mc_dropout = self.training
            
        h = self.cnn_block(x)
        if mc_dropout:
            h = F.dropout(h, p=0.3, training=True)
            
        B, C, T, Fp = h.shape
        h = h.permute(0, 2, 1, 3).contiguous().view(B, T, C*Fp)

        if lengths is not None:
            packed = nn.utils.rnn.pack_padded_sequence(h, lengths.cpu(), batch_first=True, enforce_sorted=False)
            packed_out, _ = self.rnn(packed)
            rnn_out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)
            Tmax = rnn_out.size(1)
            mask = torch.arange(Tmax, device=rnn_out.device).unsqueeze(0).expand(B, Tmax) < lengths.unsqueeze(1)
        else:
            rnn_out, _ = self.rnn(h)
            mask = torch.ones(rnn_out.size(0), rnn_out.size(1), dtype=torch.bool, device=rnn_out.device)

        if mc_dropout:
            rnn_out = F.dropout(rnn_out, p=0.3, training=True)

        rnn_out = self.rnn_ln(rnn_out)

        a = self.att(rnn_out).squeeze(-1)
        a = a.masked_fill(~mask, float('-inf'))
        w = torch.softmax(a, dim=1).unsqueeze(-1)

        mean = torch.sum(w * rnn_out, dim=1)
        var = torch.sum(w * (rnn_out - mean.unsqueeze(1))**2, dim=1)
        std = torch.sqrt(var + 1e-5)
        stats = torch.cat([mean, std], 1)

        if mc_dropout:
            stats = F.dropout(stats, p=0.3, training=True)

        z = self.proj(stats)
        z = F.normalize(z, p=2, dim=1)
        return z


class AAMSoftmax(nn.Module):
    """AAMSoftmax head (needed for loading checkpoint)."""
    def __init__(self, in_features, out_features, s=30.0, m=0.20):
        super().__init__()
        self.s = s
        self.m = m
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.empty(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, emb, labels):
        W = F.normalize(self.weight, dim=1)
        cos_theta = emb @ W.T
        theta = torch.acos(cos_theta.clamp(-1+1e-7, 1-1e-7))
        target_logits = torch.cos(theta + self.m)
        one_hot = F.one_hot(labels, num_classes=W.size(0)).float()
        output = cos_theta * (1 - one_hot) + target_logits * one_hot
        return output * self.s


class SpeakerClassifier(nn.Module):
    """Full model (loads checkpoint, but we only export backbone)."""
    def __init__(self, backbone, num_speakers, aamsm_scaler, aamsm_margin):
        super().__init__()
        self.backbone = backbone
        self.aamsm = AAMSoftmax(backbone.proj[-1].out_features, num_speakers, aamsm_scaler, aamsm_margin)
        self._inference_prepared = False
        self.score_alpha = nn.Parameter(torch.tensor(1.0))
        self.score_beta = nn.Parameter(torch.tensor(0.0))
        self.bank = None
        self.inference_threshold = 0.5

    def forward(self, x, labels=None, lengths=None):
        emb = self.backbone(x, lengths=lengths, mc_dropout=None)
        if labels is not None:
            logits = self.aamsm(emb, labels)
            return logits, emb
        return emb


print("✅ Model architecture defined")

## Section 3: Load Model and Checkpoint

Load the trained model weights from the checkpoint file.

In [ ]:
# ============================================================================
# SECTION 3: LOAD MODEL AND CHECKPOINT
# ============================================================================

def get_yaml_meta(h5_path):
    """Read metadata from HDF5."""
    with h5py.File(h5_path, "r") as f:
        raw = f["/meta/file_description.yaml"][()].decode("utf-8")
        meta_yaml = yaml.safe_load(raw) or {}
        meta_grp = f["/meta"]
        attrs = {k: (int(v) if isinstance(v, (np.integer,)) else v) for k, v in meta_grp.attrs.items()}
        meta_yaml.update(attrs)
        if "speaker_mapping.yaml" in meta_grp:
            sm_raw = meta_grp["speaker_mapping.yaml"][()].decode("utf-8")
            sm = yaml.safe_load(sm_raw) or {}
            meta_yaml["total_speakers"] = len(sm.get("speakers", {}))
    return meta_yaml


# Load metadata to get model dimensions
meta = get_yaml_meta(dataset_path)
no_mels = meta["preprocessing_config"]["n_mels"]
num_classes = meta.get("num_classes", meta["total_speakers"])

print("="*70)
print("📊 MODEL CONFIGURATION")
print("="*70)
print(f"   Mel Bands: {no_mels}")
print(f"   Num Classes: {num_classes}")

# Create model
backbone = Backbone(
    no_mels=no_mels, 
    embed_dim=256, 
    rnn_hidden=256, 
    rnn_layers=2, 
    bidir=True
)
model = SpeakerClassifier(
    backbone, 
    num_speakers=num_classes, 
    aamsm_scaler=30.0, 
    aamsm_margin=0.25
)

# Load checkpoint
print(f"\n📥 Loading checkpoint: {checkpoint_path}")
try:
    state_dict = torch.load(checkpoint_path, map_location="cpu", weights_only=True)
    model.load_state_dict(state_dict)
    print("✅ Checkpoint loaded successfully!")
except FileNotFoundError:
    print(f"❌ ERROR: Checkpoint not found!")
    print("   Please update 'checkpoint_path' above.")
except RuntimeError as e:
    print(f"❌ ERROR: {e}")

model.eval()
print("✅ Model set to evaluation mode")

## Section 4: Export to ONNX

The export process:
1. **Move model to CPU** - Standard practice for ONNX export
2. **Create dummy inputs** - ONNX uses tracing, so it needs example inputs
3. **Define dynamic axes** - Allow variable batch size and sequence length
4. **Export backbone only** - The AAMSoftmax head is not needed at inference

In [ ]:
# ============================================================================
# SECTION 4: EXPORT TO ONNX
# ============================================================================

def export_to_onnx(model, onnx_path, n_mels, dummy_time_steps=94):
    """
    Export the model's backbone to ONNX format.

    Args:
        model: SpeakerClassifier with trained weights
        onnx_path: Output path for ONNX file
        n_mels: Number of mel bands (must match training)
        dummy_time_steps: Representative time steps (94 ≈ 3 seconds at default settings)
    
    Notes:
        - We export only the BACKBONE (not AAMSoftmax)
        - Dynamic axes allow variable batch sizes and sequence lengths
        - The model outputs L2-normalized embeddings of shape [B, 256]
    """
    # Move to CPU for export
    device = torch.device("cpu")
    model = model.to(device)
    model.eval()

    print("="*70)
    print("📦 ONNX EXPORT")
    print("="*70)
    
    # Create dummy inputs
    # Input shape: [Batch, Channel=1, Time, Frequency=n_mels]
    dummy_input_x = torch.randn(1, 1, dummy_time_steps, n_mels, device=device)
    dummy_lengths = torch.tensor([dummy_time_steps], dtype=torch.long, device=device)

    print(f"   Dummy input shape: {dummy_input_x.shape}")
    print(f"   Dummy lengths: {dummy_lengths}")

    # Define input/output names
    input_names = ["input_spectrogram", "input_lengths"]
    output_names = ["embedding"]

    # Dynamic axes for flexibility
    dynamic_axes = {
        "input_spectrogram": {
            0: "batch_size",     # Variable batch
            2: "time_steps"      # Variable sequence length
        },
        "input_lengths": {
            0: "batch_size"
        },
        "embedding": {
            0: "batch_size"
        },
    }

    print(f"\n   Exporting to: {onnx_path}")
    
    # Export
    torch.onnx.export(
        model.backbone,                    # Only the backbone!
        (dummy_input_x, dummy_lengths),    # Example inputs
        onnx_path,
        verbose=False,
        input_names=input_names,
        output_names=output_names,
        dynamic_axes=dynamic_axes,
        opset_version=14,                  # Use opset 14 for better compatibility
        do_constant_folding=True,          # Optimize constants
    )
    
    # Verify file was created
    if os.path.exists(onnx_path):
        file_size = os.path.getsize(onnx_path) / (1024 * 1024)
        print(f"\n✅ ONNX export successful!")
        print(f"   File: {onnx_path}")
        print(f"   Size: {file_size:.2f} MB")
    else:
        print(f"\n❌ Export failed - file not created")
    
    return onnx_path


# Run export
exported_path = export_to_onnx(model, onnx_output_path, no_mels)

## Section 5: Verify ONNX Model

Validate the exported ONNX model by running a test inference.

In [ ]:
# ============================================================================
# SECTION 5: VERIFY ONNX MODEL
# ============================================================================

try:
    import onnx
    import onnxruntime as ort
    
    print("="*70)
    print("🔍 VERIFYING ONNX MODEL")
    print("="*70)
    
    # Load and check the ONNX model
    onnx_model = onnx.load(onnx_output_path)
    onnx.checker.check_model(onnx_model)
    print("✅ ONNX model structure is valid")
    
    # Print model inputs/outputs
    print("\n📥 Model Inputs:")
    for inp in onnx_model.graph.input:
        shape = [dim.dim_value if dim.dim_value else dim.dim_param 
                 for dim in inp.type.tensor_type.shape.dim]
        print(f"   • {inp.name}: {shape}")
    
    print("\n📤 Model Outputs:")
    for out in onnx_model.graph.output:
        shape = [dim.dim_value if dim.dim_value else dim.dim_param 
                 for dim in out.type.tensor_type.shape.dim]
        print(f"   • {out.name}: {shape}")
    
    # Test inference with ONNX Runtime
    print("\n🧪 Testing inference with ONNX Runtime...")
    session = ort.InferenceSession(onnx_output_path)
    
    # Create test input
    test_input = np.random.randn(1, 1, 94, no_mels).astype(np.float32)
    test_lengths = np.array([94], dtype=np.int64)
    
    # Run inference
    outputs = session.run(
        None, 
        {
            "input_spectrogram": test_input,
            "input_lengths": test_lengths
        }
    )
    
    embedding = outputs[0]
    print(f"   Input shape: {test_input.shape}")
    print(f"   Output shape: {embedding.shape}")
    print(f"   Embedding norm: {np.linalg.norm(embedding):.4f} (should be ~1.0)")
    
    # Compare with PyTorch output
    print("\n🔄 Comparing with PyTorch output...")
    model.eval()
    with torch.no_grad():
        torch_input = torch.from_numpy(test_input)
        torch_lengths = torch.from_numpy(test_lengths)
        torch_output = model.backbone(torch_input, torch_lengths).numpy()
    
    diff = np.abs(embedding - torch_output).max()
    print(f"   Max difference: {diff:.6f}")
    
    if diff < 1e-4:
        print("   ✅ Outputs match! ONNX export is correct.")
    else:
        print("   ⚠️ Small differences detected (may be due to floating point precision)")
    
    print("\n" + "="*70)
    print("✅ ONNX VERIFICATION COMPLETE")
    print("="*70)

except ImportError:
    print("⚠️ Install onnx and onnxruntime for verification:")
    print("   pip install onnx onnxruntime")

## Section 6: Export Speaker Bank

For inference, you also need the speaker reference embeddings (bank). Export them alongside the ONNX model.

In [ ]:
# ============================================================================
# SECTION 6: EXPORT SPEAKER BANK
# ============================================================================

print("="*70)
print("📦 EXPORTING SPEAKER BANK")
print("="*70)

# Get the speaker bank from AAMSoftmax weights
model.eval()
speaker_bank = F.normalize(model.aamsm.weight, dim=1).detach().cpu().numpy()

label_mode = meta.get("label_mode", "speaker")
print(f"   Label mode: {label_mode}")
print(f"   Speaker bank shape: {speaker_bank.shape}")

if label_mode == "binary":
    print("   → Binary mode: class 0 = outsider, class 1 = group_member")
else:
    print("   → Speaker ID mode: each row is a speaker embedding")

# Save as numpy file
bank_path = onnx_output_path.replace(".onnx", "_speaker_bank.npy")
np.save(bank_path, speaker_bank)
print(f"\n✅ Speaker bank saved to: {bank_path}")

# Also save speaker mapping if available
export_meta = {}
if "speaker_mapping" in meta:
    export_meta["speaker_mapping"] = meta["speaker_mapping"]
export_meta["label_mode"] = label_mode
export_meta["num_classes"] = speaker_bank.shape[0]
export_meta["embed_dim"] = speaker_bank.shape[1]

mapping_path = onnx_output_path.replace(".onnx", "_metadata.yaml")
with open(mapping_path, "w") as f:
    yaml.dump(export_meta, f, default_flow_style=False)
print(f"✅ Metadata saved to: {mapping_path}")

print("\n" + "="*70)
print("📁 EXPORTED FILES SUMMARY")
print("="*70)
print(f"   1. ONNX Model:   {onnx_output_path}")
print(f"   2. Speaker Bank: {bank_path}")
print(f"   3. Metadata:     {mapping_path}")
print(f"\n   Mode: {'Binary (member/outsider)' if label_mode == 'binary' else 'Speaker ID (multi-class)'}")

## Section 7: How to Use the ONNX Model

Example code for using the exported ONNX model in different environments.

In [ ]:
# ============================================================================
# SECTION 7: USAGE EXAMPLE
# ============================================================================
# This is a complete example of how to use the exported ONNX model

usage_code = '''
# ============================================================================
# PYTHON USAGE EXAMPLE
# ============================================================================

import numpy as np
import onnxruntime as ort
import librosa

# 1. Load the ONNX model and speaker bank
session = ort.InferenceSession("speaker_backbone.onnx")
speaker_bank = np.load("speaker_backbone_speaker_bank.npy")

# 2. Preprocess audio to log-mel spectrogram
def preprocess_audio(audio_path, sr=16000, n_mels=64, n_fft=2048, hop_length=512):
    """Convert audio file to log-mel spectrogram."""
    y, sr = librosa.load(audio_path, sr=sr, mono=True)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    logmel = librosa.power_to_db(mel, ref=np.max)
    
    # Shape: [1, 1, T, F]
    logmel = logmel.T[np.newaxis, np.newaxis, :, :]
    return logmel.astype(np.float32)

# 3. Run inference
def verify_speaker(audio_path, threshold=0.7):
    """Verify if speaker is authorized."""
    # Preprocess
    spectrogram = preprocess_audio(audio_path)
    lengths = np.array([spectrogram.shape[2]], dtype=np.int64)
    
    # Get embedding
    outputs = session.run(None, {
        "input_spectrogram": spectrogram,
        "input_lengths": lengths
    })
    embedding = outputs[0]
    
    # Compare to speaker bank (cosine similarity)
    similarities = embedding @ speaker_bank.T
    best_idx = np.argmax(similarities)
    best_score = similarities[0, best_idx]
    
    # Decision
    is_authorized = best_score >= threshold
    
    return {
        "speaker_id": int(best_idx),
        "confidence": float(best_score),
        "authorized": bool(is_authorized)
    }

# Example usage
result = verify_speaker("test_audio.wav")
print(f"Speaker ID: {result['speaker_id']}")
print(f"Confidence: {result['confidence']:.4f}")
print(f"Access: {'GRANTED' if result['authorized'] else 'DENIED'}")
'''

print("="*70)
print("📖 USAGE EXAMPLE (Python with ONNX Runtime)")
print("="*70)
print(usage_code)

# Save usage example to file
usage_path = onnx_output_path.replace(".onnx", "_usage_example.py")
with open(usage_path, "w") as f:
    f.write(usage_code)
print(f"\n✅ Usage example saved to: {usage_path}")

## 📚 Summary

### Files Exported:
| File | Description |
|------|-------------|
| `speaker_backbone.onnx` | The neural network model (backbone only) |
| `speaker_backbone_speaker_bank.npy` | Reference embeddings for each speaker |
| `speaker_backbone_speaker_mapping.yaml` | Speaker ID to name mapping |
| `speaker_backbone_usage_example.py` | Python example code |

### Inference Pipeline:
1. **Preprocess**: Audio → Log-mel spectrogram
2. **Embed**: Spectrogram → ONNX model → 256-dim embedding
3. **Match**: Cosine similarity with speaker bank
4. **Decide**: Compare max similarity to threshold

### Deployment Options:
- **Python**: `onnxruntime` (CPU or GPU)
- **C++**: ONNX Runtime C++ API
- **JavaScript**: `onnxruntime-web` for browser
- **Mobile**: ONNX Runtime for iOS/Android
- **Edge**: TensorRT, OpenVINO for optimization